# 서울지도를 이용한 시각화

- 데이터 : 생활인구 데이터 (서울 열린데이터 광장) https://data.seoul.go.kr/dataVisual/seoul/seoulLivingPopulation.do
- 생활인구란? 조사시점 머물고 있는 현재 인구

In [1]:
import pandas as pd
import folium
from folium import plugins
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import geopandas as gpd

fontpath='/usr/share/fonts/truetype/nanum/NanumBarunGothic.tff'
font=fm.FontProperties(fname=fontpath,size=9)
plt.rc('font',family='NanumBarunGothic')
mpl.font_manager._rebuild()

### 1) 데이터 불러오기

In [2]:
local=pd.read_csv('LOCAL_PEOPLE_20200513.csv',engine='python', encoding='euc-kr')
local.head()

,"?""기준일ID""",시간대구분,행정동코드,집계구코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자25세부터29세생활인구수,여자30세부터34세생활인구수,여자35세부터39세생활인구수,여자40세부터44세생활인구수,여자45세부터49세생활인구수,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수
0,20200513,8,11110560,1101056030003,35.3457,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
1,20200513,15,11110560,1101056030003,37.6209,*,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
2,20200513,15,11260600,1107061020001,574.4813,42.1455,16.7482,15.6067,19.3199,14.6589,...,21.5807,23.1868,18.8165,18.2772,16.68,19.8903,19.0246,19.3208,15.4995,56.5716
3,20200513,22,11260600,1107061020001,723.2091,31.9927,12.7136,11.4994,17.9117,25.1092,...,34.12,41.0414,27.902,22.0213,28.6304,28.8468,30.1962,22.6266,15.1406,60.5972
4,20200513,5,11260600,1107061020002,449.9735,13.6301,5.4165,7.6755,18.3191,13.833,...,23.8328,21.8214,13.2091,16.9302,19.9703,14.0453,25.4704,9.6071,11.1375,38.5124


- local의 행정동코드와 code의 행자부행정동 코드가 같다는 것을 알 수 있다

In [3]:
code=pd.read_csv("동_코드.csv",engine='python', encoding='euc-kr')
code.head()

,통계청행정동코드,행자부행정동코드,시도명,시군구명,행정동명
0,1101053,11110530,서울,종로구,사직동
1,1101054,11110540,서울,종로구,삼청동
2,1101055,11110550,서울,종로구,부암동
3,1101056,11110560,서울,종로구,평창동
4,1101057,11110570,서울,종로구,무악동


- 행정동의 위치가 있는 json파일

In [4]:
state_geo = 'HangJeongDong.json'
geo_str=json.load(open(state_geo,encoding='utf-8'))
geo_str

{'type': 'FeatureCollection',
 'name': 'HangJeongDong_ver20200515',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'OBJECTID': 0,
    'adm_nm': '서울특별시 종로구 사직동',
    'adm_cd': '1101053',
    'adm_cd2': '1111053000',
    'sgg': '11110',
    'sido': '11',
    'sidonm': '서울특별시',
    'sggnm': '종로구'},
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[126.97688884274817, 37.575650779448786],
       [126.9770344988775, 37.569194530054546],
       [126.97597472821249, 37.569336299425764],
       [126.97537470991254, 37.56931556702156],
       [126.97433193562325, 37.56926180051753],
       [126.96904837001854, 37.56819441770833],
       [126.96854493603384, 37.56842767961276],
       [126.9666499598212, 37.569491655206576],
       [126.96628175024485, 37.5697007347987],
       [126.9660973270804, 37.5698565097237],
       [126.96572852922577, 37.570183936115114],
       [126.96592699822128, 37

- json형식을 csv형식으로 보기
- code 테이블의 통계청행정동코드와 adm_cd가 동일하다는 것을 알 수 있다.

In [5]:
geo_str_df=gpd.read_file('HangJeongDong.json',driver='GeoJSON')
geo_str_df=geo_str_df[["adm_nm",'adm_cd','geometry']]
geo_str_df

,adm_nm,adm_cd,geometry
0,서울특별시 종로구 사직동,1101053,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,서울특별시 종로구 삼청동,1101054,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,서울특별시 종로구 부암동,1101055,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,서울특별시 종로구 평창동,1101056,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,서울특별시 종로구 무악동,1101057,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...
3482,경기도 의정부시 송산3동,3103071,"MULTIPOLYGON (((127.10534 37.74080, 127.10327 ..."
3483,경기도 용인시기흥구 영덕1동,3119263,"MULTIPOLYGON (((127.09981 37.27007, 127.09933 ..."
3484,경기도 용인시기흥구 상갈동,3119265,"MULTIPOLYGON (((127.12676 37.26603, 127.12538 ..."
3485,경기도 용인시기흥구 동백2동,3119268,"MULTIPOLYGON (((127.15189 37.27909, 127.15327 ..."


## 2) 전처리과정

- local과 code 테이블을 보면, 행정동코드가 존재한다. 
- 이 코드를 통해 실제 시도, 군구, 동명을 함께 불러오기 위해 type를 str로 변경해준다.

In [6]:
local['행정동코드']=local['행정동코드'].astype(str)
code['행자부행정동코드']=code['행자부행정동코드'].astype(str)
code['통계청행정동코드']=code['통계청행정동코드'].astype(str)
geo_str_df['adm_cd']=geo_str_df['adm_cd'].astype(str) 

In [7]:
local_people = pd.merge(local, code, how='inner',left_on='행정동코드', right_on='행자부행정동코드')
len(local_people)

459652

- 오류2동의 통계청행정동코드와 매칭이 되지않은 것을 알 수 있다.
- 이것은 직접 변경한다.

In [8]:
a=pd.merge(local_people, geo_str_df , how='left',left_on='통계청행정동코드', right_on='adm_cd')
a[a['adm_cd'].isna()]["행정동명"].unique()

array(['오류2동'], dtype=object)

- json 파일의 코드: 1117073
- csv 파일의 코드: '1117068'
- 2020.01.01 이후 json파일에 있는 코드로 변경되어 오류2동 변경!

In [9]:
geo_str_df[geo_str_df["adm_nm"].str.endswith("오류2동")]

,adm_nm,adm_cd,geometry
3478,서울특별시 구로구 오류2동,1117073,"MULTIPOLYGON (((126.83179 37.47757, 126.83235 ..."


In [10]:
local_people[local_people["행정동명"]=="오류2동"]["통계청행정동코드"].unique()

array(['1117068'], dtype=object)

In [11]:
local_people.loc[local_people["행정동명"] == "오류2동", "통계청행정동코드"] = "1117073" 
local_people["통계청행정동코드"]=local_people["통계청행정동코드"].astype(str)
local_people.loc[local_people["행정동명"] == "오류2동"].head()

,"?""기준일ID""",시간대구분,행정동코드,집계구코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,...,여자50세부터54세생활인구수,여자55세부터59세생활인구수,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수,통계청행정동코드,행자부행정동코드,시도명,시군구명,행정동명
293288,20200513,2,11530780,1117068010001,348.4827,28.36,11.1778,16.4639,8.3588,9.1339,...,11.5239,16.9279,11.1636,8.1076,19.7576,1117073,11530780,서울,구로구,오류2동
293289,20200513,9,11530780,1117068010001,505.9265,25.3714,9.9999,20.4535,16.2619,15.6105,...,19.5694,26.989,19.0678,15.3163,31.5692,1117073,11530780,서울,구로구,오류2동
293290,20200513,16,11530780,1117068010001,672.8635,37.5482,14.7993,24.2188,11.4085,17.4005,...,25.4317,27.1723,24.2843,21.0398,38.1529,1117073,11530780,서울,구로구,오류2동
293291,20200513,23,11530780,1117068010001,348.4512,25.728,10.1405,15.4051,4.875,9.5875,...,12.2091,14.5409,11.1523,10.3835,22.3813,1117073,11530780,서울,구로구,오류2동
293292,20200513,6,11530780,1117068010002,760.0505,27.9229,11.0056,17.1229,21.2063,28.8581,...,22.3309,39.5277,23.1046,20.0348,42.209,1117073,11530780,서울,구로구,오류2동


- 항동: 1117074 이지만, 오류2동 "1117073" 로 변경해서 같이 취급하겠다.

In [12]:
geo_str_df.loc[geo_str_df["adm_cd"]=='1117074',"adm_cd"]= "1117073" 
geo_str_df[geo_str_df["adm_cd"]=='1117073']

,adm_nm,adm_cd,geometry
270,서울특별시 구로구 항동,1117073,"MULTIPOLYGON (((126.82405 37.48885, 126.82478 ..."
3478,서울특별시 구로구 오류2동,1117073,"MULTIPOLYGON (((126.83179 37.47757, 126.83235 ..."


- 생활인구는 시간대별로 다양하기때문에 행정동 기준으로 평균을 냈다.
- json파일과 생활인구 데이터를 merge한다. 
- 이 때, 통계청행정동코드를 기준으로 한다.

In [13]:
nbh_count_df= local_people[['통계청행정동코드',"행정동명","총생활인구수"]].groupby(['통계청행정동코드',"행정동명"],as_index=False).mean()
nbh_geo_count_df = pd.merge(geo_str_df, nbh_count_df, left_on='adm_cd',  right_on='통계청행정동코드')
nbh_geo_count_df

,adm_nm,adm_cd,geometry,통계청행정동코드,행정동명,총생활인구수
0,서울특별시 종로구 사직동,1101053,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ...",1101053,사직동,1670.201620
1,서울특별시 종로구 삼청동,1101054,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ...",1101054,삼청동,1225.876593
2,서울특별시 종로구 부암동,1101055,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ...",1101055,부암동,762.064492
3,서울특별시 종로구 평창동,1101056,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ...",1101056,평창동,504.943168
4,서울특별시 종로구 무악동,1101057,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ...",1101057,무악동,449.202430
...,...,...,...,...,...,...
420,서울특별시 강동구 둔촌1동,1125070,"MULTIPOLYGON (((127.14580 37.52199, 127.14564 ...",1125070,둔촌1동,190.434284
421,서울특별시 강동구 둔촌2동,1125071,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ...",1125071,둔촌2동,526.593601
422,서울특별시 강동구 암사1동,1125072,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ...",1125072,암사1동,388.039118
423,서울특별시 강동구 천호2동,1125073,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ...",1125073,천호2동,736.710283


- 현재 서울시 행정동은 425개이지만, 오류2동 = 항동을 하여 424개
- 서울에 424 개의 행정동이 존재함을 알 수 있다
- 오류2동 -> 오류2동과 항동으로 나눠지면서 데이터를 구할 수 없다

In [14]:
nbh_geo_count_df[nbh_geo_count_df["통계청행정동코드"]=='1117073']

,adm_nm,adm_cd,geometry,통계청행정동코드,행정동명,총생활인구수
270,서울특별시 구로구 항동,1117073,"MULTIPOLYGON (((126.82405 37.48885, 126.82478 ...",1117073,오류2동,578.614263
271,서울특별시 구로구 오류2동,1117073,"MULTIPOLYGON (((126.83179 37.47757, 126.83235 ...",1117073,오류2동,578.614263
